In [20]:
import argparse
import io
import os
import sys
import easydict

import dfareporting_utils
from googleapiclient import http
from oauth2client import client

In [25]:
Declare command-line flags.
argparser = argparse.ArgumentParser(add_help=False)
argparser.add_argument(
    '112473040', type=int,
    help='The ID of the report to get a file for')
argparser.add_argument(
    '698165673', type=int,
    help='The ID of the file to get')



# Chunk size to use when downloading report files. Defaults to 32MB.
CHUNK_SIZE = 32 * 1024 * 1024

AttributeError: 'set' object has no attribute 'items'

In [14]:
def main(argv):
  # Retrieve command line arguments.
  flags = dfareporting_utils.get_arguments(argv, __doc__, parents=[argparser])

  # Authenticate and construct service.
  service = dfareporting_utils.setup(flags)

  report_id = flags.report_id
  file_id = flags.file_id

  try:
    # Retrieve the file metadata.
    report_file = service.files().get(reportId=report_id,
                                      fileId=file_id).execute()

    if report_file['status'] == 'REPORT_AVAILABLE':
      # Prepare a local file to download the report contents to.
      out_file = io.FileIO(generate_file_name(report_file), mode='wb')

      # Create a get request.
      request = service.files().get_media(reportId=report_id, fileId=file_id)

      # Create a media downloader instance.
      # Optional: adjust the chunk size used when downloading the file.
      downloader = http.MediaIoBaseDownload(out_file, request,
                                            chunksize=CHUNK_SIZE)

      # Execute the get request and download the file.
      download_finished = False
      while download_finished is False:
        _, download_finished = downloader.next_chunk()

      print('File %s downloaded to %s'
            % (report_file['id'], os.path.realpath(out_file.name)))

  except client.AccessTokenRefreshError:
    print ('The credentials have been revoked or expired, please re-run the '
           'application to re-authorize')

In [15]:
def generate_file_name(report_file):
  """Generates a report file name based on the file metadata."""
  # If no filename is specified, use the file ID instead.
  file_name = report_file['fileName'] or report_file['id']
  extension = '.csv' if report_file['format'] == 'CSV' else '.xml'
  return file_name + extension

In [16]:
if __name__ == '__main__':
  main(sys.argv)

usage: ipykernel_launcher.py [-h] [--auth_host_name AUTH_HOST_NAME]
                             [--noauth_local_webserver]
                             [--auth_host_port [AUTH_HOST_PORT [AUTH_HOST_PORT ...]]]
                             [--logging_level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
                             112473040 698165673
ipykernel_launcher.py: error: argument 112473040: invalid int value: 'C:\\Users\\cesar.requena\\AppData\\Roaming\\jupyter\\runtime\\kernel-0093a03b-c226-4f51-aa91-138ca9a66f8e.json'


SystemExit: 2

C:\Users\cesar.requena\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
